In [2]:
import geopandas as gpd
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

from shapely import geometry, ops
from shapely.geometry import MultiLineString, LineString, Point
import os
import concurrent.futures
from shapely.ops import unary_union
from shapely.geometry import JOIN_STYLE, Polygon, MultiPolygon

import pycountry
import xyt

import time

### Ajuster le module TP pour livraison aux cantons

In [3]:
# IMPORT DATA
user_stat = pd.read_pickle('../Data/processed_feuille_de_route/gps_user_statistics.pkl')

legs = pd.read_pickle('../Data/processed_feuille_de_route/legs_nogeometry.pkl')

module_tp = pd.read_csv("../Data/dumps_fors/EPFL Panel Lémanique GPS - Données Users & TimeTables/EPFL Panel Lémanique GPS - TimeTables.csv")

module_tp = pd.merge(module_tp, user_stat[['user_id_fors','user_id_motiontag']].drop_duplicates(),
                    how = 'left', left_on = 'IDNO', right_on = 'user_id_fors')

module_tp.head(3)

,storyline_id,storyline_uuid,departure_at,departure_station_id,departure_station_name,departure_station_latitude,departure_station_longitude,arrival_at,arrival_station_id,arrival_station_name,arrival_station_latitude,arrival_station_longitude,mode,line_number,train_number,operator,IDNO,user_id_fors,user_id_motiontag
0,177078904,4567a555-c104-4b16-a495-04f96c2033a6,2023-04-24 15:18:00,8516155:0:1,Lancy-Pont-Rouge,46.185943,6.124947,2023-04-24 15:33:00,8774549,Annemasse,46.199344,6.236401,regional_train,RE,Annemasse,Schweizerische Bundesbahnen SBB,FR2513,FR2513,45c9c698-e26c-4ee9-abbc-99a1aac4b214
1,177087460,432b57eb-b36f-44a0-abc7-ab04d98f3e0d,2023-04-24 15:39:00,8579254,"Lausanne, St-François",46.519365,6.633483,2023-04-24 15:54:00,8592189,"Pully, Moulins",46.508410,6.670009,bus,9,"Lutry, Corniche",Transports Publics de la Région Lausannoise sa,CH183,CH183,f1dfbd77-25e5-4004-8dea-dfc38aa57d4d
2,177086487,5332a798-6080-420f-b355-01ac9602b724,2023-04-24 15:47:00,8592802,"Genève, Charmilles",46.208278,6.127337,2023-04-24 15:51:00,8592890,"Genève, Prairie",46.208496,6.135745,bus,19,"Onex, cité",Transports Publics Genevois,CH2294,CH2294,fc56e9a9-19fb-4b77-bd37-3b69275a73bc


In [12]:
user_stat

,user_id_fors,user_id_motiontag,KT_home_survey,KT_home_gps,first_activity_date,last_activity_date,days_in_range,active_days_count,days_without_event,days_with_track,days_without_track,home_geometry_from_gps,home_geometry_from_survey,ID_municipality_survey,ID_Agglo_survey,N_Agglo_survey,gdr,prof,age_fr,wgt_cant_gps,wgt_agg_gps,wgt_cant_trim_gps,wgt_agg_trim_gps,main_motor,car_in_HH_count
0,CH3181,000ca786-ebb5-4f48-bfb4-68022b6f7f7b,VD,VD,2023-04-30,2023-06-05,37,33,4,29,4,POINT (6.53632 46.73237),POINT (6.53629 46.73239),5757,6153,Monthey,Femme,Population active occupée,30-44 ans,0.296419,0.304223,0.296419,0.304223,essence,1.0
1,CH7869,001807b0-d393-47f1-be7d-a28708ab38b8,VD,VD,2023-05-04,2023-06-05,33,18,15,-1,19,POINT (6.59279 46.54076),POINT (6.59211 46.54074),5410,6153,Monthey,Femme,Population active occupée,60-74 ans,0.906823,0.972201,0.906823,0.972201,essence,1.0
2,CH23573,0022452b-97e0-4e5f-b01e-1168e92364e6,NaN,VD,2023-04-24,2023-06-01,39,39,0,39,0,POINT (6.63667 46.78205),None,NaN,99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,CH1007,00284b1d-cc9c-46a7-9877-d5ab9aad7fd4,VD,NaN,2023-05-02,2023-05-31,30,23,6,17,6,POINT (8.22003 46.19169),POINT (6.61431 46.53175),5586,6153,Monthey,Femme,Population active occupée,30-44 ans,0.301482,0.323217,0.301482,0.323217,NaN,0.0
4,CH26652,00367e58-ad00-41db-b93b-1f1e55c5a733,VD,VD,2023-05-11,2023-06-05,26,20,6,14,6,POINT (6.74324 46.79801),POINT (6.74555 46.79948),5939,6153,Monthey,Homme,Population active occupée,18-29 ans,0.455058,0.467038,0.455058,0.467038,essence,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2799,CH8750,ffaa5d6b-288b-46ed-800d-ef8e4ab42512,GE,GE,2023-04-25,2023-06-02,39,36,3,33,3,POINT (6.17954 46.19239),POINT (6.17954 46.19220),6621,6621,Genève,Homme,Population active occupée,30-44 ans,0.887434,0.995842,0.887434,0.995842,essence,4.0
2800,CH19818,ffb32fa8-79c7-4360-9025-552fb8ca149b,VD,VD,2023-05-02,2023-06-02,32,32,0,32,0,POINT (6.95434 46.32195),POINT (6.97057 46.31903),5401,6153,Monthey,Femme,Population active occupée,30-44 ans,0.607275,0.447643,0.607275,0.447643,essence,2.0
2801,CH23315,ffc1f825-f35b-4065-97c0-f761856c142d,VD,VD,2023-04-24,2023-06-03,41,38,3,35,3,POINT (6.57776 46.62315),POINT (6.57733 46.62799),5471,6153,Monthey,Homme,Personnes en formation,18-29 ans,0.648241,0.665307,0.648241,0.665307,essence,3.0
2802,FR12202,ffe06c6e-3ddb-4e3f-ae40-1506e85a16c7,FRA,FRA,2023-05-10,2023-06-05,27,21,6,15,6,POINT (6.62502 46.39833),POINT (6.62512 46.39822),74200,99,FRA,Femme,Population active occupée,30-44 ans,0.123956,0.204727,0.123956,0.204727,diesel,2.0


In [25]:
#Check if storyline_uuid eq to leg_id
uuids = module_tp.storyline_uuid.sample(5)
legs.loc[legs.leg_id.isin(uuids)]

,leg_id,user_id_day,user_id_fors,type,legs_date,started_at,started_at_timezone,finished_at,finished_at_timezone,length,detected_mode,mode,purpose,confirmed_at,started_on,misdetected_completely,merged,created_at,updated_at,started_at_in_timezone,finished_at_in_timezone,confirmed_at_in_timezone,created_at_in_timezone,updated_at_in_timezone,point_per_linestring,max_signlalloss_meters,length_leg,rel_max_signalloss,low_quality_legs_1,low_quality_legs_2,usr_w_constant_bad_signal,leading_stay_id,duration,activity_in_KT,panel_area,KT_home_survey
200420,4ea6bf1e-6e46-4c04-bdbf-b7c4744ddce9,CH20441_20230427,CH20441,Track,2023-04-27,2023-04-27 04:58:52,Europe/Zurich,2023-04-27 05:10:27,Europe/Zurich,4044.410360,Mode::Bus,Mode::Bus,NaN,2023-04-27 16:51:55.586,2023-04-27,f,f,2023-04-27 05:17:54.824,2023-04-27 16:51:55.586,2023-04-27T06:58:52+02:00,2023-04-27T07:10:27+02:00,2023-04-27T16:51:55Z,2023-04-27T05:17:54Z,2023-04-27T16:51:55Z,322,221.480066,4044.410357,0.054762,0,0,0,NaN,695.0,NaN,NaN,NaN
250598,fd48370d-fe4d-44c4-807b-ed19074cb8a4,CH22901_20230517,CH22901,Track,2023-05-17,2023-05-17 14:08:30,Europe/Zurich,2023-05-17 14:17:54,Europe/Zurich,2783.502314,Mode::Subway,Mode::Subway,NaN,NaN,2023-05-17,f,f,2023-05-17 14:33:41.247,2023-05-17 14:33:41.247,2023-05-17T16:08:30+02:00,2023-05-17T16:17:54+02:00,NaN,2023-05-17T14:33:41Z,2023-05-17T14:33:41Z,70,231.316337,2783.502312,0.083103,0,0,0,NaN,564.0,NaN,NaN,NaN
443416,7628d4d6-5f77-4b14-8a71-5e74a5690970,CH5165_20230524,CH5165,Track,2023-05-24,2023-05-24 06:51:55,Europe/Zurich,2023-05-24 07:01:15,Europe/Zurich,1853.490484,Mode::Tram,Mode::Tram,NaN,NaN,2023-05-24,f,f,2023-05-24 08:08:13.952,2023-05-24 08:08:13.952,2023-05-24T08:51:55+02:00,2023-05-24T09:01:15+02:00,NaN,2023-05-24T08:08:13Z,2023-05-24T08:08:13Z,138,131.247116,1853.490483,0.070811,0,0,0,NaN,560.0,NaN,NaN,NaN
455862,c36f8176-f3c3-4240-a75d-0c96b28fd1c2,CH5888_20230520,CH5888,Track,2023-05-20,2023-05-20 07:10:51,Europe/Zurich,2023-05-20 08:56:58,Europe/Zurich,154523.665213,Mode::Train,Mode::Train,NaN,NaN,2023-05-20,f,f,2023-05-20 09:16:17.744,2023-05-20 09:16:17.744,2023-05-20T09:10:51+02:00,2023-05-20T10:56:58+02:00,NaN,2023-05-20T09:16:17Z,2023-05-20T09:16:17Z,2787,2013.772361,154523.665107,0.013032,0,0,0,NaN,6367.0,NaN,NaN,NaN
502997,615da75f-b857-416b-b953-f5f6ecc90f36,CH8806_20230525,CH8806,Track,2023-05-25,2023-05-25 15:42:56,Europe/Zurich,2023-05-25 15:48:16,Europe/Zurich,1261.309110,Mode::Bus,Mode::Bus,NaN,2023-05-27 13:52:55.671,2023-05-25,f,f,2023-05-25 15:50:21.492,2023-05-27 13:52:55.671,2023-05-25T17:42:56+02:00,2023-05-25T17:48:16+02:00,2023-05-27T13:52:55Z,2023-05-25T15:50:21Z,2023-05-27T13:52:55Z,49,111.899584,1261.309108,0.088717,0,0,0,NaN,320.0,NaN,NaN,NaN


In [26]:
# CHANGER L'ORDRE DES COLONNES ET SUPPRIMER 'storyline_id'

sort_cols = ['user_id_fors','user_id_motiontag','storyline_uuid', 'departure_at',
       'departure_station_id', 'departure_station_name',
       'departure_station_latitude', 'departure_station_longitude',
       'arrival_at', 'arrival_station_id', 'arrival_station_name',
       'arrival_station_latitude', 'arrival_station_longitude', 'mode',
       'line_number', 'train_number', 'operator']

module_tp['departure_at'] = pd.to_datetime(module_tp['departure_at'])
module_tp_ = module_tp[sort_cols].sort_values(by=['user_id_fors','departure_at']).copy().reset_index(drop=True).rename(columns={'storyline_uuid':'leg_id'})
module_tp_.head(2)

,user_id_fors,user_id_motiontag,leg_id,departure_at,departure_station_id,departure_station_name,departure_station_latitude,departure_station_longitude,arrival_at,arrival_station_id,arrival_station_name,arrival_station_latitude,arrival_station_longitude,mode,line_number,train_number,operator
0,CH01,be168a66-975b-4558-98e8-524c04352039,6eaf3044-d637-46ed-b755-ac81ac33ed3a,2023-05-16 14:50:00,8592204,"Renens VD, Bois-Genoud",46.546303,6.584265,2023-05-16 14:52:00,8588842,"Crissier, Bré",46.550479,6.579270,bus,18,"Crissier, Timonet",Transports Publics de la Région Lausannoise sa
1,CH01,be168a66-975b-4558-98e8-524c04352039,d4b6b329-a629-491a-b47a-6b87d3d785c6,2023-05-19 22:40:00,8501200:0:2,Vevey,46.463004,6.843447,2023-05-19 22:55:00,8501400:0:1,Aigle,46.316857,6.963686,train,IR90,Sion,Schweizerische Bundesbahnen SBB


In [27]:
# EXPORTER LE NUOVEAU FICHIER
#module_tp_.to_csv('../Data/processed_feuille_de_route/240528_module_tp.csv')

### Ajuster le fichier Dépacement pour livraison aux cantons

In [ ]:
# IMPORT DATA
trips = pd.read_csv("../Data/dumps_motiontag/Trips2023-04-01--2023-08-31.csv")
trips.rename(columns={'id':'trip_id','user_id':'user_id_motiontag'},inplace=True)
trips.head(3)


In [ ]:
trips_ = pd.merge(user_stat[['user_id_fors','user_id_motiontag']], trips, how='left', on='user_id_motiontag')
trips_.head(3)

In [ ]:
# CHECK THE MERGE
len(trips) - len(trips_)

In [ ]:
#We loose some data because user_stat discarded some users from the original motiontag dump (probably bad users)
len(trips.loc[~trips['trip_id'].isin(trips_.trip_id.unique()), 'user_id_motiontag'].unique())

In [ ]:
# EXPORTER LE NUOVEAU FICHIER
#trips_.to_csv('../Data/processed_feuille_de_route/240528_trips.csv')

### Parse Legs from pkl to csv

In [ ]:
#legs.to_csv('../Data/donnees_livrees/20240528_dump_situee/20240329_legs_nogeometry.csv')